# Set up

In [38]:
%matplotlib inline
%run fcast.py

# customize matplotlib plots
mpl.style.use('seaborn-darkgrid')
plt.rc('font', family='TH Sarabun New')
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes',  labelsize=16)
plt.rc('figure', figsize=(6.5,4), dpi=300)

# Prepare Data

In [39]:
hml = read_price('hml')
hml.meta = SimpleNamespace()
hml.meta.abb = 'hml'
hml.meta.shortname = 'ข้าวหอมมะลิ'
hml.meta.fullname = 'ราคาข้าวเปลือกหอมมะลิ ที่เกษตรกรขายได้ ณ ไร่นา'

wht = read_price('wht')
wht.meta = SimpleNamespace()
wht.meta.abb = 'wht'
wht.meta.shortname = 'ข้าวเจ้าขาว'
wht.meta.fullname = 'ราคาข้าวเปลือกเจ้าขาว ที่เกษตรกรขายได้ ณ ไร่นา'

glu = read_price('glu')
glu.meta = SimpleNamespace()
glu.meta.abb = 'glu'
glu.meta.shortname = 'ข้าวเหนียว'
glu.meta.fullname = 'ราคาข้าวเปลือกเหนียวเมล็ดยาว ที่เกษตรกรขายได้ ณ ไร่นา'

prices = [hml, wht, glu]

In [40]:
for price in prices:
#   price.scaled, price.scaler = data_transform(price, method="log")
  price.scaled, price.scaler = data_transform(price, method=None)
  price.train, price.validate, price.train2, price.test = data_split(price.scaled)
  print(price.meta.abb, price.train.size, price.validate.size, price.test.size)

hml 259 85 85
wht 259 85 85
glu 259 85 85


# ARIMA

## Generate Parameters Grid

In [41]:
p = list(range(3))
q = list(range(3))
P = [0, 1]
Q = [0, 1]
t = ['c']
m = 12
order = model_configs(p, [1], q)
sorder = model_configs(P, [0], Q, [12])
arima_configs = model_configs(order, sorder, t)
arima_configs

[((0, 1, 0), (0, 0, 0, 12), 'c'),
 ((0, 1, 0), (0, 0, 1, 12), 'c'),
 ((0, 1, 0), (1, 0, 0, 12), 'c'),
 ((0, 1, 0), (1, 0, 1, 12), 'c'),
 ((0, 1, 1), (0, 0, 0, 12), 'c'),
 ((0, 1, 1), (0, 0, 1, 12), 'c'),
 ((0, 1, 1), (1, 0, 0, 12), 'c'),
 ((0, 1, 1), (1, 0, 1, 12), 'c'),
 ((0, 1, 2), (0, 0, 0, 12), 'c'),
 ((0, 1, 2), (0, 0, 1, 12), 'c'),
 ((0, 1, 2), (1, 0, 0, 12), 'c'),
 ((0, 1, 2), (1, 0, 1, 12), 'c'),
 ((1, 1, 0), (0, 0, 0, 12), 'c'),
 ((1, 1, 0), (0, 0, 1, 12), 'c'),
 ((1, 1, 0), (1, 0, 0, 12), 'c'),
 ((1, 1, 0), (1, 0, 1, 12), 'c'),
 ((1, 1, 1), (0, 0, 0, 12), 'c'),
 ((1, 1, 1), (0, 0, 1, 12), 'c'),
 ((1, 1, 1), (1, 0, 0, 12), 'c'),
 ((1, 1, 1), (1, 0, 1, 12), 'c'),
 ((1, 1, 2), (0, 0, 0, 12), 'c'),
 ((1, 1, 2), (0, 0, 1, 12), 'c'),
 ((1, 1, 2), (1, 0, 0, 12), 'c'),
 ((1, 1, 2), (1, 0, 1, 12), 'c'),
 ((2, 1, 0), (0, 0, 0, 12), 'c'),
 ((2, 1, 0), (0, 0, 1, 12), 'c'),
 ((2, 1, 0), (1, 0, 0, 12), 'c'),
 ((2, 1, 0), (1, 0, 1, 12), 'c'),
 ((2, 1, 1), (0, 0, 0, 12), 'c'),
 ((2, 1, 1), (

## Define functions
- arima_fit
- arima_forecast
- arima_walk_forward

In [42]:
def arima_fit(data, config):
  order, sorder, trend = config
  return pm.arima.ARIMA(order, sorder, trend=trend).fit(data)

def arima_forecast(model_fitted, data, n_forecast=4):
  scaler = getattr(data, "scaler")
  df = pd.DataFrame(model_fitted.predict(n_forecast),
                    columns=["yhat_scaled"])
  df['yhat'] = scaler.inverse_transform(df)
  return df['yhat']

def arima_walk_forward(data, config, search_mode=False, n_forecast=4):
  train, test = select_traintest(data, search_mode) # this ensures that we use transformed train and test set
  data_ = pd.concat([train,test])
  N = data_.shape[0]
  n_test = test.shape[0]
  n_yhat = n_test+1
  yhat = np.empty((n_yhat, n_forecast))
  
  history = train
  model_fitted = arima_fit(history, config)
  yhat[0] = arima_forecast(model_fitted, data, n_forecast)
  
  for i in range(1, n_yhat):
    model_fitted = model_fitted.update(test.iloc[i-1])
    yhat[i] = arima_forecast(model_fitted, data, n_forecast)
    history = history.append(test.iloc[i-1])

  yhat = pd.DataFrame(yhat,
                      columns=[f'yhat{h}' for h in range(1, n_forecast+1)],
                      index=data_.index[-n_test-1:])
  
  return yhat

In [6]:
# test
# config = arima_configs[0]
# fitted = arima_fit(hml.scaled, config)
# yhat = arima_walk_forward(hml, config, search_mode=True)
# scores = model_measure(hml, yhat, config)
# scores

# Run

In [43]:
arima_model = [arima_fit, arima_forecast, arima_walk_forward, arima_configs]

In [44]:
%%time
n_forecast=4
for price in prices:
  price.arima_grid_result, price.arima_config = grid_search(price, arima_model, n_forecast=n_forecast)
  price.arima_yhat = forecast(price, arima_model, price.arima_config, n_forecast=n_forecast)

{('(0, 1, 0)-(0, 0, 0, 12)-c',): {'rmse': 1229.6174, 'mae': 770.3659, 'mape': 0.0592}}
{('(0, 1, 0)-(0, 0, 1, 12)-c',): {'rmse': 1231.4258, 'mae': 776.6997, 'mape': 0.0598}}
{('(0, 1, 0)-(1, 0, 0, 12)-c',): {'rmse': 1237.182, 'mae': 781.7487, 'mape': 0.0602}}
{('(0, 1, 0)-(1, 0, 1, 12)-c',): {'rmse': 1221.6116, 'mae': 742.3321, 'mape': 0.0557}}
{('(0, 1, 1)-(0, 0, 0, 12)-c',): {'rmse': 1210.9134, 'mae': 761.1787, 'mape': 0.0584}}
{('(0, 1, 1)-(0, 0, 1, 12)-c',): {'rmse': 1217.2168, 'mae': 767.9952, 'mape': 0.0591}}
{('(0, 1, 1)-(1, 0, 0, 12)-c',): {'rmse': 1220.3141, 'mae': 771.9332, 'mape': 0.0593}}
{('(0, 1, 1)-(1, 0, 1, 12)-c',): {'rmse': 1209.2541, 'mae': 738.5686, 'mape': 0.0555}}
{('(0, 1, 2)-(0, 0, 0, 12)-c',): {'rmse': 1256.3924, 'mae': 777.7636, 'mape': 0.0596}}
{('(0, 1, 2)-(0, 0, 1, 12)-c',): {'rmse': 1262.2919, 'mae': 785.9427, 'mape': 0.0603}}
{('(0, 1, 2)-(1, 0, 0, 12)-c',): {'rmse': 1259.6974, 'mae': 786.4878, 'mape': 0.0603}}
{('(0, 1, 2)-(1, 0, 1, 12)-c',): {'rmse': 12

In [29]:
path = '../results/result_arima_4.pkl'

if(os.path.isfile(path)):
  result = joblib.load(path)
else:
  result = dict()
  for price in prices:
    result[price.meta.abb] = [price.arima_grid_result, price.arima_config, price.arima_yhat]
  joblib.dump(result, path)

In [48]:
hml.arima_yhat

,yhat1,yhat2,yhat3,yhat4
date,,,,
2012-08-01,"15,627.5546","15,688.0918","15,435.5700","15,356.2783"
2012-09-01,"15,573.0491","15,320.9654","15,241.7526","15,394.9658"
2012-10-01,"15,627.8403","15,548.5102","15,701.8353","15,866.1266"
2012-11-01,"15,131.4928","15,285.3011","15,451.0186","15,494.3591"
2012-12-01,"15,610.8328","15,776.5691","15,819.9102","16,033.2883"
...,...,...,...,...
2019-05-01,"15,754.3627","15,842.2736","15,892.5239","15,998.0029"
2019-06-01,"15,863.7887","15,914.0841","16,019.7704","15,989.8713"
2019-07-01,"15,456.4519","15,561.5067","15,531.3728","15,184.9046"


In [22]:
# plot_fcast(hml, 'arima', step=2)

In [45]:
model_measure(hml, hml.arima_yhat, [""])

,rmse,mae,mape,rmse1,mae1,mape1,rmse2,mae2,mape2,rmse3,mae3,mape3,rmse4,mae4,mape4
,828.4682,602.5464,0.0489,445.7445,303.5510,0.0247,738.5302,540.2320,0.0441,963.1385,702.6554,0.0572,"1,166.4594",863.7471,0.0698


In [36]:
model_measure(hml, hml.arima_yhat, [""])

,rmse,mae,mape,rmse1,mae1,mape1,rmse2,mae2,mape2,rmse3,mae3,mape3,rmse4,mae4,mape4
,926.3782,670.5727,0.0545,478.2798,329.7268,0.0269,827.1463,593.1281,0.0482,"1,089.1806",784.3278,0.0638,"1,310.9061",975.1080,0.0791


In [46]:
model_measure(wht, wht.arima_yhat, [""])

,rmse,mae,mape,rmse1,mae1,mape1,rmse2,mae2,mape2,rmse3,mae3,mape3,rmse4,mae4,mape4
,489.3947,381.9047,0.0480,289.2557,213.8977,0.0262,452.2483,364.1154,0.0454,569.6511,442.9919,0.0558,646.4236,506.6138,0.0644


In [32]:
model_measure(wht, wht.arima_yhat, [""])

,rmse,mae,mape,rmse1,mae1,mape1,rmse2,mae2,mape2,rmse3,mae3,mape3,rmse4,mae4,mape4
,497.9089,382.2650,0.0476,288.7314,214.5631,0.0262,461.7035,373.2554,0.0464,581.4132,450.1570,0.0563,659.7876,491.0845,0.0616


In [47]:
model_measure(glu, glu.arima_yhat, [""])

,rmse,mae,mape,rmse1,mae1,mape1,rmse2,mae2,mape2,rmse3,mae3,mape3,rmse4,mae4,mape4
,853.0905,636.2395,0.0578,516.9155,363.2587,0.0332,784.4453,564.4327,0.0515,966.1163,748.0140,0.0681,"1,144.8850",869.2527,0.0782


In [34]:
model_measure(glu, glu.arima_yhat, [""])

,rmse,mae,mape,rmse1,mae1,mape1,rmse2,mae2,mape2,rmse3,mae3,mape3,rmse4,mae4,mape4
,863.8839,645.9053,0.0592,514.8356,362.1703,0.0333,796.3268,584.7623,0.0537,980.6102,752.2634,0.0691,"1,163.7630",884.4251,0.0806


In [35]:
glu.arima_yhat

,yhat1,yhat2,yhat3,yhat4
date,,,,
2012-08-01,"12,866.2134","12,930.1530","13,005.0259","13,076.6438"
2012-09-01,"13,117.6838","13,192.9085","13,249.5596","13,303.9318"
2012-10-01,"12,129.8658","12,186.5095","12,323.6249","12,445.1796"
2012-11-01,"11,703.1681","11,837.0428","11,994.7870","12,107.6135"
2012-12-01,"12,137.3899","12,297.8262","12,387.3318","12,438.0946"
...,...,...,...,...
2019-05-01,"11,524.5142","11,497.7074","11,445.0252","11,432.0437"
2019-06-01,"11,927.1349","11,865.7238","11,814.9049","11,814.6358"
2019-07-01,"11,881.6433","11,830.5444","11,828.8829","11,864.1621"
